MBTdelay -- model_LinearRegression.ipynb

© Mark Mace 2019 markfmace@gmail.com

Performs linear regression based machine learning on MBTDelay datasets

In [1]:
# GENERAL INCLUSIONS
import numpy as np
import glob,os
import matplotlib.pyplot as plt
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

import pickle

from sklearn import metrics
from sklearn.metrics import mean_squared_error, r2_score

from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate

from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import RandomizedSearchCV

from sklearn.pipeline import Pipeline

# FOR DATES AND TIMES #
import time
import datetime
from dateutil import tz
from datetime import timedelta
import arrow

In [2]:
# CONVERT UNIX (utc) TIMESTAMP TO YYYY-mm-dd HH:MM:SS (EASTERN)
def conv_unixts_to_east_hms(ts):
    east=arrow.Arrow.fromtimestamp(ts).to('US/Eastern')
    return east.format('YYYY-MM-DD HH:mm:ss')

def get_hour(dt_str):
    return dt_str[11:13]

def get_month_num(dt_str):
    return dt_str[5:7]


# RETURNS DAY OF WEEK 
# M-0, Tu-1, W-2, Th-3 F-4 Sa-6 Su-7
# TAKES YYYY-MM-DD HH:MM:SS RETURNS Day (IN WHATEVER TIMEZONE)
def get_day_of_week(dt):
    dtt = arrow.get(dt)
    return dtt.weekday()

# TAKES UNIX TS RETURNS Day (IN EASTERN/BOSTON)
def get_day_of_week_east_unix(ts):
    east=conv_unixts_to_east_hms(ts)
    return get_day_of_week(east)
    
# TAKES UNIX TS RETURNS Day (IN UTC)
def get_day_of_week_utc_unix(ts):
    utc=conv_unixts_to_utc_hms(ts)
    return get_day_of_week(utc)
    


In [19]:
### ML SPECIFIC FUNCTIONS
### FOR FEATURE ENGINEERING
# ONE HOT ENCODE EVENT -- 1--event, 0--no event
def bin_event(x):
    x=int(x)
    if(x!=0):
        return 1
    else:
        return 0

# YES OR NO 
def bin_weather(x):
    x=float(x)
    if(x>0):
        return 1
    else:
        return 0
    
# BIN PRECIPITATION TYPE
def bin_ptype(x):
    if(x==1): # None
        return 0
    else: # rain, snow or sleet
        return 0
    
# BIN IN d_bin SECONDS
d_bin=10
def bin_delay(x):
    x=float(x)
    if(x<=0):
        return 0
    else:
        return int(x/d_bin)
    
#BIN IN t_bin DEGREES
t_bin=10
def bin_temp(x):
    x=float(x)
    if(x<=0):
        return 0
    else:
        return int(x/t_bin)
    
        
# PEAK HOUR BIN
def bin_peak(x):
    x=float(x)
    if(6<=x<=10 or 4<=x<=7):
        return 1
    else:
        return 0
           
# WEEKDAY BIN
def bin_weekday(x):
    x=float(x)
    if(x<5):
        return 1 # WEEKDAY
    else:
        return 0 # WEEKEND
    
# SEASON
def bin_season(x):
    x=float(x)
    if(x in {1,2,12}):
        return 0 # WINTER
    elif(x in {3,4,5}):
        return 1 # SPRING
    elif(x in {9,10,11}):
        return 2 # FALL
    elif(x in {6,7,8}):
        return 3 # SUMMER
    else:
        print("NOT A VALID MONTH")
        return -1 # WRONG

In [20]:
# another self-defined eval metric
# f(y_true: array, y_pred: array) -> name: string, eval_result: float, is_higher_better: bool
# Relative Absolute Error (RAE)
def rae(y_true, y_pred):
    return 'RAE', np.sum(np.abs(y_pred - y_true)) / np.sum(np.abs(np.mean(y_true) - y_true)), False


In [21]:
MBTdelay_df=[]
ds_files=glob.glob("DS/DS_*.csv")
for ds_f in ds_files:
#     print(ds_f)
    station_name,station_id=ds_f.replace("DS/DS_","").replace(".csv","").split("_")
    df_temp=pd.read_csv(ds_f)

    MBTdelay_df.append([station_name,station_id,df_temp])
    
    # ENCODE PRECIP TYPE AS NUMBER
MBTdelay_df=np.array(MBTdelay_df)
MBTdelay_df[0][2].columns

Index(['Unnamed: 0', 'U_DATETIME', 'HEAD_GAP', 'HDW_T', 'BNCH_HDW_T', 'TEMP',
       'PRECIP_INT', 'PRECIP_PRO', 'PRECIP_ACC', 'PRECIP_TYP', 'u_datetime',
       'event', 'period', 'DN', 'DATETIME', 'HOUR_BIN', 'MONTH_BIN', 'DOW',
       'PRECIP_TYP_ENC'],
      dtype='object')

In [22]:
# RESULTS OF THE FUNCTION BELOW #
f_combo_1=[0,0,1,0,1,0,1,0,0]
f_combo_2=[0,1,1,0,1,0,0,0,0] # <-- THIS IS THE BEST COMBO

# FEATURE ENGINEER DATA AND PERFORM TRAIN-TEST SPLIT
def feature_engineer_and_split(loc_df,EVENT_FLAG,SNOW_YN_FLAG,RAIN_YN_FLAG,
          WEEKDAY_FLAG,PRECIP_TYP_FLAG,PEAK_FLAG,DELAY_FLAG,SEASON_FLAG,TEMP_FLAG,split_perc,shuffle_flag):
    
#     print(np.mean(test_df['HEAD_GAP']))
    fe_df=loc_df[np.isfinite(loc_df['TEMP'])].copy()

    if(EVENT_FLAG==1):
        fe_df['event']=fe_df['event'].apply(lambda x: bin_weather(x))

    if(SNOW_YN_FLAG==1):
        fe_df['PRECIP_ACC']=fe_df['PRECIP_ACC'].apply(lambda x: bin_weather(x))

    if(RAIN_YN_FLAG==1):
        fe_df['PRECIP_INT']=fe_df['PRECIP_INT'].apply(lambda x: bin_weather(x))

    if(WEEKDAY_FLAG==1):
        fe_df['DOW']=fe_df['DOW'].apply(lambda x: bin_weekday(x))
        
    if(PRECIP_TYP_FLAG==1):
        fe_df['PRECIP_TYP_ENC']=fe_df['PRECIP_TYP_ENC'].apply(lambda x: bin_ptype(x))

    if(PEAK_FLAG==1):
        fe_df['HOUR_BIN']=fe_df['HOUR_BIN'].apply(lambda x: bin_peak(x))

    if(DELAY_FLAG==1):
        fe_df['HEAD_GAP']=fe_df['HEAD_GAP'].apply(lambda x: bin_delay(x))

    if(SEASON_FLAG==1):
        fe_df['MONTH_BIN']=fe_df['MONTH_BIN'].apply(lambda x: bin_season(x))
        
    if(TEMP_FLAG==1):
        fe_df['TEMP']=fe_df['TEMP'].apply(lambda x: bin_temp(x))

    # NOT INCLUDING PRECIP TYPE #
    all_x=np.transpose(np.array([fe_df['HOUR_BIN'], fe_df['DOW'], fe_df['MONTH_BIN'],
                                fe_df['event'],
                                fe_df['PRECIP_INT'], fe_df['PRECIP_ACC'], 
                                fe_df['PRECIP_PRO'], fe_df['TEMP']],dtype=np.float32))

    all_y=np.array([fe_df['HEAD_GAP']],dtype=np.float32)[0]


    X_train, X_test, y_train, y_test = train_test_split(all_x, all_y, test_size=split_perc, random_state=0,shuffle=shuffle_flag)
    
    return X_train, X_test, y_train, y_test 


In [25]:
## THIS IS JUST FOR TESTING ##

# TEST STATION NUMBER # 
station_num_id=3

test_size=0.2

# LOCAL LOAD DATAFRAME 
station_name=MBTdelay_df[station_num_id][0]
station_id=MBTdelay_df[station_num_id][1]
test_df=MBTdelay_df[station_num_id][2].copy()
test_df = test_df[np.isfinite(test_df['HEAD_GAP'])]

X_train, X_test, y_train, y_test=feature_engineer_and_split(test_df, 0,1,1,0,1,0,1,0,0,test_size,True)
# X_train, X_test, y_train, y_test=feature_engineer_and_split(df_loc,0,1,1,0,1,0,1,0,0,0.2) # FEATURE ENGINEERED
# Create linear regression object
regr = linear_model.LinearRegression()

# Train the model using the training sets
regr.fit(X_train, y_train)

print('Saving model...')
# save model to file
pickle.dump(regr, open('NEW_LR_MODEL/'+station_name+'_'+station_id+'.pk', 'wb'))
    

print('Starting predicting...')

# Make predictions using the testing set
y_pred = regr.predict(X_test)

# eval
print('The rmse of prediction is:', mean_squared_error(y_test, y_pred) ** 0.5)
print('The rae of prediction is:', rae(y_test, y_pred)[1])
print('Variance score: ',r2_score(y_test,y_pred))


Saving model...
Starting predicting...
The rmse of prediction is: 16.36712503717198
The rae of prediction is: 0.9742725
Variance score:  0.03435810547181506


In [24]:
# BUFFERS FOR ANALYTICS
data_metrics=[]
model_outcomes=[]

# TRAIN-TEST SPLIT
test_size=0.2

# RUN MODEL FOR ALL STATIOS AND SAVE TO FILE #
for station_num_id in range(len(MBTdelay_df)):
    
    # LOAD DATAFRAME 
    station_name=MBTdelay_df[station_num_id][0]
    station_id=MBTdelay_df[station_num_id][1]
    test_df=MBTdelay_df[station_num_id][2].copy()

    # SIMPLE STATS FOR DATA FOR A STATION
    gap_mode=stats.mode(test_df['HEAD_GAP'].values)
    gap_mean=np.mean(test_df['HEAD_GAP'])
    gap_mean_unc=np.std(test_df['HEAD_GAP'])/np.sqrt(len(test_df['HEAD_GAP']))
    
    # SAVE METRICS TO ARRAY 
    data_metrics.append([station_name,station_id,
                    gap_mode,
                    gap_mean,
                    gap_mean_unc
                    ])
    
    # GENERATE TEST-TRAIN SPLIT -- USES FEATURE ENGINEERING DETERMINED PREVIOUSLY #
    X_train, X_test, y_train, y_test=feature_engineer_and_split(test_df, 0,1,1,0,1,0,0,0,0, test_size, True)
    
    # Create linear regression object
    regr = linear_model.LinearRegression()

    # Train the model using the training sets
    regr.fit(X_train, y_train)

    print('Saving model...')
    # save model to file
    pickle.dump(regr, open('LR_MODEL/'+station_name+'_'+station_id+'.pk', 'wb'))

    print('Starting predicting...')

    # Make predictions using the testing set
    y_pred = regr.predict(X_test)

    # EVALUATE METRICS
    mae=metrics.mean_absolute_error(y_test, y_pred)
    mse=metrics.mean_squared_error(y_test, y_pred)
    rmse=np.sqrt(metrics.mean_squared_error(y_test, y_pred))
    r2=r2_score(y_test,y_pred)
    
    # SAVE MODEL METRICS
    model_outcomes.append([station_name,station_id,mae,mse,rmse,r2])

model_outcomes=np.array(model_outcomes)

mo_df=pd.DataFrame({'station_name': model_outcomes[:, 0], 
                    'station_id': model_outcomes[:, 1], 
                    'mae': model_outcomes[:, 2], 
                    'mse': model_outcomes[:, 3], 
                    'rmse': model_outcomes[:, 4], 
                    'r2': model_outcomes[:, 5]})

mo_df.to_csv('LR_MODEL/Outcomes.csv')

Saving model...
Starting predicting...
Saving model...
Starting predicting...
Saving model...
Starting predicting...
Saving model...
Starting predicting...
Saving model...
Starting predicting...
Saving model...
Starting predicting...
Saving model...
Starting predicting...
Saving model...
Starting predicting...
Saving model...
Starting predicting...
Saving model...
Starting predicting...
Saving model...
Starting predicting...
Saving model...
Starting predicting...
Saving model...
Starting predicting...
Saving model...
Starting predicting...
Saving model...
Starting predicting...
Saving model...
Starting predicting...
Saving model...
Starting predicting...
Saving model...
Starting predicting...
Saving model...
Starting predicting...
Saving model...
Starting predicting...
Saving model...
Starting predicting...
Saving model...
Starting predicting...
Saving model...
Starting predicting...
Saving model...
Starting predicting...
Saving model...
Starting predicting...
Saving model...
Starting 